In [1]:
from tiki_crawler import Tiki
import time
import requests
import json
import re
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from kafka import KafkaProducer
from json import dumps
from kafka import KafkaConsumer
from json import loads

In [2]:
tiki = Tiki()

In [3]:
tiki.cat_info

[['bach-hoa-online', 4384],
 ['nha-cua-doi-song', 1883],
 ['dien-tu-dien-lanh', 4221],
 ['thiet-bi-kts-phu-kien-so', 1815],
 ['dien-thoai-may-tinh-bang', 1789],
 ['do-choi-me-be', 2549],
 ['lam-dep-suc-khoe', 1520],
 ['dien-gia-dung', 1882],
 ['thoi-trang-nu', 931],
 ['thoi-trang-nam', 915],
 ['giay-dep-nu', 1703],
 ['tui-vi-nu', 976],
 ['giay-dep-nam', 1686],
 ['tui-thoi-trang-nam', 27616],
 ['balo-va-vali', 6000],
 ['phu-kien-thoi-trang', 27498],
 ['dong-ho-va-trang-suc', 8371],
 ['laptop-may-vi-tinh-linh-kien', 1846],
 ['hang-quoc-te', 17166],
 ['voucher-dich-vu', 11312],
 ['o-to-xe-may-xe-dap', 8594],
 ['nha-sach-tiki', 8322],
 ['the-thao-da-ngoai', 1975],
 ['may-anh', 1801]]

In [4]:
s_category = 0
e_category = 1
start_page = 1
end_page = 2

In [5]:
# from pyspark.sql import SparkSession

# spark = SparkSession.\
#         builder.\
#         appName("pyspark-notebook").\
#         master("spark://spark-master:7077").\
#         config("spark.executor.memory", "512m").\
#         getOrCreate()

In [6]:
producer = KafkaProducer(bootstrap_servers=['kafka-1:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [7]:

df = []
comments = []
for i in range(s_category, e_category, 1):
    category, category_id = tiki.cat_info[i]
    for page_index in tqdm(range(start_page, end_page)):
        time.sleep(0.5)
        category_page_url = tiki.api_get_id_product.format(category_id, page_index, category)

        list_product_ids = tiki.get_list_id_product(category_page_url)
        id = 0
        if len(list_product_ids) > 0:
            for pid,spid in list_product_ids:
                id+=1
                print(id)
                product_url = tiki.api_product.format(pid,spid)
                item = tiki.get_information_product(product_url)
                producer.send('Product', value=item)
                
#                 df.append(item)
                
#                 review_url = tiki.api_review.format(1,spid,pid)
#                 paging = tiki.get_max_paging_review(review_url)
#                 print(review_url)
            
#                 if paging != 0:
#                     for i in range(1,paging+1):
# #                         time.sleep(0.2)
#                         comment = tiki.get_review(review_url)
#                         comments.append(comment)
                
                

  0%|          | 0/1 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


100%|██████████| 1/1 [00:14<00:00, 14.10s/it]
